In [2]:
import pandas as pd
from Levenshtein import distance
from metaphone import doublemetaphone
import pyspark
import findspark
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

### Load the City Directory data, only the columns relevant for the matching and disambiguation process 
- Matching will be done using ward number, enumeration district, and first and last names.
- Occupation and house number can be useful for disambiguation

In [136]:
cd_df = pd.read_csv("/Users/candide/Acads/HNYC/cd_1880_mn_v04.csv", usecols=['OBJECTID', 'CD_FIRST_NAME', \
        'CD_MIDDLE_NAME', 'CD_LAST_NAME', 'MATCH_ADDR', 'WARD_NUM', 'CD_ED', 'CD_OCCUPATION', \
        'CD_FINAL_HOUSENUM'], index_col = 0, dtype={"CD_ED":str, "WARD_NUM":str}, keep_default_na=False, \
                    na_values=['NaN'])

In [137]:
cd_df.head()

,MATCH_ADDR,CD_FIRST_NAME,CD_LAST_NAME,CD_MIDDLE_NAME,CD_OCCUPATION,CD_FINAL_HOUSENUM,WARD_NUM,CD_ED
OBJECTID,,,,,,,,
1,"10 DOMINICK ST, New York, NY",Louis,Zollie,NA,NA,10,8,111
2,"106 W 24 ST, New York, NY",Ignatz,Zoller,NA,shoes,106,16,343
3,"504 W 55 ST, New York, NY",Stephen,Zoller,NA,molder,504,22,513
4,"210 W 46 ST, New York, NY",Oscar,Zollikoffer,NA,pres,210,22,469
5,"241 W 53D ST, New York, NY",Caroline,Zollinger,NA,wid,241,22,504


In [125]:
cd_df[cd_df.CD_MIDDLE_NAME.notnull()]

,OBJECTID,MATCH_ADDR,CD_FIRST_NAME,CD_LAST_NAME,CD_MIDDLE_NAME,CD_OCCUPATION,CD_FINAL_HOUSENUM,WARD_NUM,CD_ED
0,1,"10 DOMINICK ST, New York, NY",Louis,Zollie,NA,NA,10,8,111
1,2,"106 W 24 ST, New York, NY",Ignatz,Zoller,NA,shoes,106,16,343
2,3,"504 W 55 ST, New York, NY",Stephen,Zoller,NA,molder,504,22,513
3,4,"210 W 46 ST, New York, NY",Oscar,Zollikoffer,NA,pres,210,22,469
4,5,"241 W 53D ST, New York, NY",Caroline,Zollinger,NA,wid,241,22,504
5,6,"65 W 54 ST, New York, NY",Oscar,Zollikoffer,F.,sec,65,19,582
6,7,"208 E 49 ST, New York, NY",Moses,Zollfrey,NA,cutter,208,19,550
7,8,"5 N William St, New York, NY",Robert,Zoller,NA,birds,5,4,36
8,9,"315 W 35 ST, New York, NY",Charles,Zollner,NA,express,315,20,401
9,10,"203 W 30 ST, New York, NY",Joseph,Zong,NA,driver,203,20,410


In [138]:
len(cd_df)

223436

In [139]:
## take the unique rows
cd_df.drop_duplicates(inplace=True)
len(cd_df)

222618

In [140]:
cd_df.reset_index(inplace=True)

In [5]:
cd_df.drop(columns=['CD_MIDDLE_NAME'], inplace=True)

### Load the Census data. Again, just the columns relevant for the matching and disambiguation process 
- Matching will be done using ward number, enumeration district, and first and last names.
- Age, occupation and segment id can be useful for disambiguation

In [41]:
census_df = pd.read_csv("/Users/candide/Acads/HNYC/census_1880_mn_v04.csv", usecols=['OBJECTID.x', 'CENSUS_NAMEFRSTB',
            'CENSUS_NAMELASTB', 'CENSUS_MATCH_ADDR', 'WARD_NUM', 'CENSUS_ENUMDIST', 'CENSUS_AGE', 'CENSUS_OCCLABELB',
            'CENSUS_SEGMENT_ID'], index_col=0, dtype={"CENSUS_ENUMDIST":str, "WARD_NUM":str}, keep_default_na=False, \
                    na_values=['NaN'])

/Users/candide/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [45]:
census_df.loc[1077888]

CENSUS_NAMEFRSTB                                JOHN
CENSUS_NAMELASTB                                 OTT
CENSUS_AGE                                         9
CENSUS_OCCLABELB                                   *
CENSUS_MATCH_ADDR    1994 3RD AVE, NYC-Manhattan, NY
CENSUS_SEGMENT_ID                               4796
CENSUS_ENUMDIST                                  630
WARD_NUM                                          12
Name: 1077888, dtype: object

In [46]:
len(census_df)

1117479

### Clean the names in CENSUS

In [47]:
def get_calling_name(name_string):
    mess = name_string.split(' ')
    return max(mess, key=len)

census_df['CENSUS_NAMEFRSTCLEAN'] = census_df['CENSUS_NAMEFRSTB'].apply(lambda x: get_calling_name(x))

In [38]:
census_df[['CENSUS_NAMEFRSTB', 'CENSUS_NAMEFRSTCLEAN']]

,CENSUS_NAMEFRSTB,CENSUS_NAMEFRSTCLEAN
OBJECTID.x,,
1,EDITH K.,EDITH
2,EMMA,EMMA
3,LIZZIE,LIZZIE
4,FANNIE K.,FANNIE
5,MITCHILL A.,MITCHILL
6,JOSEPH P.,JOSEPH
7,JAMES E.,JAMES
8,JAMES E.,JAMES
9,HENRY W.,HENRY


In [48]:
## take the unique rows
census_df.drop_duplicates(inplace=True)
census_df.reset_index(inplace=True)
census_df.drop(['CENSUS_NAMEFRSTB'], axis=1, inplace=True)

In [34]:
census_df.columns.values

array(['CENSUS_NAMELASTB', 'CENSUS_AGE', 'CENSUS_OCCLABELB',
       'CENSUS_MATCH_ADDR', 'CENSUS_SEGMENT_ID', 'CENSUS_ENUMDIST',
       'WARD_NUM', 'CENSUS_NAMEFRSTCLEAN'], dtype=object)

In [147]:
cd_df['WARD_ED'] = cd_df['WARD_NUM'].str.strip() + "_" + cd_df['CD_ED'].str.strip()
census_df['WARD_ED'] = census_df['WARD_NUM'].str.strip() + "_" + census_df["CENSUS_ENUMDIST"].str.strip()

In [148]:
cd_df.drop(['WARD_NUM', 'CD_ED'], axis=1, inplace=True)

In [149]:
census_df.drop(['WARD_NUM', 'CENSUS_ENUMDIST'], axis=1, inplace=True)

In [ ]:
cd_census_merged = cd_df.merge(census_df, on="WARD_ED", how='inner')

In [2]:
findspark.init()
spark = pyspark.sql.SparkSession.builder \
     .master("local") \
     .appName("yelp") \
     .config("spark.some.config.option", conf=pyspark.SparkConf()) \
     .getOrCreate()

In [31]:
census_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1117479 entries, 1 to 1117479
Data columns (total 7 columns):
CENSUS_NAMELASTB        1117479 non-null object
CENSUS_AGE              1117479 non-null int64
CENSUS_OCCLABELB        1117479 non-null object
CENSUS_MATCH_ADDR       1117479 non-null object
CENSUS_SEGMENT_ID       1117479 non-null int64
CENSUS_NAMEFRSTCLEAN    1117479 non-null object
WARD_ED                 1117479 non-null object
dtypes: int64(2), object(5)
memory usage: 68.2+ MB


### Save the pre-processed cd and census data. Will use those files in running code for linking 

In [150]:
cd_df.to_csv("cd_df_final.csv", index=False)
census_df.to_csv("census_df_final.csv", index=False)

In [151]:
del cd_df, census_df

### (You may skip over the following cells, should you find them confusing. )
### The join/merge code in the following cells is not clean, and was actually run on Google colab for computer power. See the script `'run_link_records.ipynb'` for the final version of this code.

In [152]:
cd_df_pys = spark.read.csv("cd_df_final_last.csv", header=True)
census_df_pys = spark.read.csv("census_df_final.csv", header=True)

In [111]:
def levenshteinMetaphoneFunc(first_cd, first_cn, last_cd, last_cn):
    first_cd, last_cd, first_cn, last_cn = first_cd.strip(), last_cd.strip(), first_cn.strip(), last_cn.strip() 
    s1 = (first_cd + last_cd).lower()
    s2 = (first_cn + last_cn).lower()
    mf_first_cd = set(doublemetaphone(first_cd))
    mf_last_cd = set(doublemetaphone(last_cd))
    mf_first_cn = set(doublemetaphone(first_cn))
    mf_last_cn = set(doublemetaphone(last_cn))
    if len(mf_first_cd.intersection(mf_first_cn).difference({''})) > 0:
        first_good = True
    else:
        first_good = False
    if len(mf_last_cd.intersection(mf_last_cn).difference({''})) > 0:
        last_good = True
    else:
        last_good = False
    if (distance(s1, s2) < 4) and first_good and last_good:
        return True
    return False

In [112]:
from pyspark.sql.types import BooleanType
func = spark.udf.register("nameMatchCheckWithPython", levenshteinMetaphoneFunc, BooleanType())

In [153]:
merged = cd_df_pys.join(census_df_pys, 'WARD_ED', 'inner')
results = merged.filter(func(merged.CD_FIRST_NAME, merged.CENSUS_NAMEFRSTCLEAN, merged.CD_LAST_NAME, \
                          merged.CENSUS_NAMELASTB))

In [156]:
results.repartition(1).write.csv('cd_census_merged_last_22.csv', mode="append", header="true")

In [ ]:
cd_df_pys_ = spark.read.csv("cd_df_final_15_22.csv", header=True)
merged = cd_df_pys_.join(census_df_pys, 'WARD_ED', 'inner')
results = merged.filter(func(merged.CD_FIRST_NAME, merged.CENSUS_NAMEFRSTCLEAN, merged.CD_LAST_NAME, \
                          merged.CENSUS_NAMELASTB))
results.repartition(1).write.csv('cd_census_merged_last_15_22.csv', mode="append", header="true")

In [97]:
name = tuple(cd_df.sample()[['CD_FIRST_NAME', 'CD_LAST_NAME']].iloc[0,:])

In [114]:
names = [('Charles', 'Wolfe'), ('John', 'Roller'), ('John', 'Abbott'), ('Jacob', 'Abraham'), \
         ('Nathan', 'Abrams'), ('Peter', 'Albrecht'), ('Simon', 'Goodman')]

In [152]:
cd_df[(cd_df.CD_FIRST_NAME.str.contains('Abbott')) & \
                      (cd_df.CD_LAST_NAME.str.contains(''))][['CD_FIRST_NAME', \
                        'CD_LAST_NAME', 'CD_ED', 'WARD_NUM', 'MATCH_ADDR']]

,CD_FIRST_NAME,CD_LAST_NAME,CD_ED,WARD_NUM,MATCH_ADDR


In [150]:
census_df[census_df.CENSUS_NAMELASTB.str.contains(''.upper()) & \
              census_df.CENSUS_NAMEFRSTB.str.contains('Abbott'.upper())][['CENSUS_NAMEFRSTB', \
               'CENSUS_NAMELASTB', 'CENSUS_ED', 'WARD_NUM', 'MATCH_ADDR']]

,CENSUS_NAMEFRSTB,CENSUS_NAMELASTB,CENSUS_ED,WARD_NUM,MATCH_ADDR


In [ ]:
for name in names:
    result_cd = cd_df[(cd_df.CD_FIRST_NAME.str.contains(name[0])) & \
                      (cd_df.CD_LAST_NAME.str.contains(name[1]))][['CD_FIRST_NAME', \
                        'CD_LAST_NAME', 'CD_ED', 'WARD_NUM', 'MATCH_ADDR']]
    result_census = census_df[census_df.CENSUS_NAMELASTB.str.contains(name[1].upper()) & \
              census_df.CENSUS_NAMEFRSTB.str.contains(name[0].upper())][['CENSUS_NAMEFRSTB', \
               'CENSUS_NAMELASTB', 'CENSUS_ED', 'WARD_NUM', 'MATCH_ADDR']]
    print(result_cd.to_string(index=False))
    print(result_census.to_string(index=False))
    print('\n')

In [ ]:
census_df_sample = census_df.head(100)
result_census = census_df[census_df.apply(lambda x: levenshteinDistanceAllowed("CharlesWolfer", x), axis=1)][['CENSUS_NAMEFRSTB','CENSUS_NAMELASTB', \
                                                                'CENSUS_ED', 'WARD_NUM', 'MATCH_ADDR']]
# print(result_cd.to_string(index=False))
print(result_census.to_string(index=False))
print('\n')

## A demonstration of the *doublementaphone( )* results

In [175]:
doublemetaphone("Smith")

('SM0', 'XMT')

In [171]:
doublemetaphone("Schmidt")

('XMT', 'SMT')

In [81]:
if 1 & len(a.intersection(b).difference({''})):
    print("yes")
else:
    print("no")

yes


In [82]:
len(a.intersection(b).difference({''}))

1

In [83]:
a.intersection(b).difference({''})

{'SKLR'}

In [2]:
df1 = pd.read_csv("~/Acads/HNYC/cd_df_final.csv")

In [9]:
df1.head()

,OBJECTID,MATCH_ADDR,CD_FIRST_NAME,CD_LAST_NAME,CD_MIDDLE_NAME,CD_OCCUPATION,CD_FINAL_HOUSENUM,WARD_ED
0,1,"10 DOMINICK ST, New York, NY",Louis,Zollie,NaN,NaN,10,8_111
1,2,"106 W 24 ST, New York, NY",Ignatz,Zoller,NaN,shoes,106,16_343
2,3,"504 W 55 ST, New York, NY",Stephen,Zoller,NaN,molder,504,22_513
3,4,"210 W 46 ST, New York, NY",Oscar,Zollikoffer,NaN,pres,210,22_469
4,5,"241 W 53D ST, New York, NY",Caroline,Zollinger,NaN,wid,241,22_504


### Reording the columns for a more sensible order

In [13]:
df1 = df1[['OBJECTID', 'CD_FIRST_NAME', 'CD_LAST_NAME', 'CD_MIDDLE_NAME', 'CD_OCCUPATION', 'MATCH_ADDR', 
           'CD_FINAL_HOUSENUM', 'WARD_ED']]

In [14]:
df1.head()

,OBJECTID,CD_FIRST_NAME,CD_LAST_NAME,CD_MIDDLE_NAME,CD_OCCUPATION,MATCH_ADDR,CD_FINAL_HOUSENUM,WARD_ED
0,1,Louis,Zollie,NaN,NaN,"10 DOMINICK ST, New York, NY",10,8_111
1,2,Ignatz,Zoller,NaN,shoes,"106 W 24 ST, New York, NY",106,16_343
2,3,Stephen,Zoller,NaN,molder,"504 W 55 ST, New York, NY",504,22_513
3,4,Oscar,Zollikoffer,NaN,pres,"210 W 46 ST, New York, NY",210,22_469
4,5,Caroline,Zollinger,NaN,wid,"241 W 53D ST, New York, NY",241,22_504


In [16]:
df2 = pd.read_csv("/Users/candide/Acads/HNYC/census_df_final.csv")

In [17]:
df2.head()

,OBJECTID.x,CENSUS_NAMELASTB,CENSUS_AGE,CENSUS_OCCLABELB,CENSUS_MATCH_ADDR,CENSUS_SEGMENT_ID,CENSUS_NAMEFRSTCLEAN,WARD_ED
0,1,SANGER,1,*,"36 GOVERNORS ISLAND, NYC-Manhattan, NY",2,EDITH,1_1
1,2,VAN BROSELIN,27,DOMESTIC SERVANT,"36 GOVERNORS ISLAND, NYC-Manhattan, NY",2,EMMA,1_1
2,3,MALONE,27,DOMESTIC SERVANT,"36 GOVERNORS ISLAND, NYC-Manhattan, NY",2,LIZZIE,1_1
3,4,SANGER,28,NONE,"36 GOVERNORS ISLAND, NYC-Manhattan, NY",2,FANNIE,1_1
4,5,LEE,28,DOMESTIC SERVANT,"36 GOVERNORS ISLAND, NYC-Manhattan, NY",2,MITCHILL,1_1


In [18]:
df2 = df2[['OBJECTID.x', 'CENSUS_NAMEFRSTCLEAN', 'CENSUS_NAMELASTB', 'CENSUS_AGE', 'CENSUS_OCCLABELB',
       'CENSUS_MATCH_ADDR', 'CENSUS_SEGMENT_ID', 'WARD_ED']]

In [19]:
df2.head()

,OBJECTID.x,CENSUS_NAMEFRSTCLEAN,CENSUS_NAMELASTB,CENSUS_AGE,CENSUS_OCCLABELB,CENSUS_MATCH_ADDR,CENSUS_SEGMENT_ID,WARD_ED
0,1,EDITH,SANGER,1,*,"36 GOVERNORS ISLAND, NYC-Manhattan, NY",2,1_1
1,2,EMMA,VAN BROSELIN,27,DOMESTIC SERVANT,"36 GOVERNORS ISLAND, NYC-Manhattan, NY",2,1_1
2,3,LIZZIE,MALONE,27,DOMESTIC SERVANT,"36 GOVERNORS ISLAND, NYC-Manhattan, NY",2,1_1
3,4,FANNIE,SANGER,28,NONE,"36 GOVERNORS ISLAND, NYC-Manhattan, NY",2,1_1
4,5,MITCHILL,LEE,28,DOMESTIC SERVANT,"36 GOVERNORS ISLAND, NYC-Manhattan, NY",2,1_1


In [20]:
df1.to_csv("cd_df_final.csv", index=False)
df2.to_csv("census_df_final.csv", index=False)